## Unencrypted connections
- Num of channels sending at least one unencrypted HTTP request
- Percentage of unencrypted HTTP requests

In [1]:
import pandas as pd
from df_utils import load_df, print_stats
from log_analysis import get_playback_detection_results, get_n_successful_channels
from nb_utils import get_popular_domains_from_reqs, get_popular_domains_from_tcp_conns, make_latex_table
from crawl_ids import CrawlRokuTop1KMITM, CrawlFireTVTop1KMITM

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Total num. of channels

### Load Requests and TCP connections

In [9]:
requests_roku = load_df(CrawlRokuTop1KMITM, "http_req")
requests_roku = requests_roku[(~requests_roku.url.str.startswith("https://"))]
# (requests_roku.tcp_dstport!=443) & 
responses_roku = load_df(CrawlRokuTop1KMITM, "http_resp")
dns_df_roku = load_df(CrawlRokuTop1KMITM, "dns")

requests_amazon = load_df(CrawlFireTVTop1KMITM, "http_req")
requests_amazon = requests_amazon[(~requests_amazon.url.str.startswith("https://"))]

responses_amazon = load_df(CrawlFireTVTop1KMITM, "http_resp")
dns_df_amazon = load_df(CrawlFireTVTop1KMITM, "dns")

In [5]:
roku_tcp = load_df(CrawlRokuTop1KMITM, "tcp_conn")
amazon_tcp = load_df(CrawlFireTVTop1KMITM, "tcp_conn")

In [10]:
print("%s Roku channels send at last one unencrypted request" % (requests_roku.channel_id.nunique()))
print("%s Fire TV channels send at last one unencrypted request" % (requests_amazon.channel_id.nunique()))

814 Roku channels send at last one unencrypted request
891 Fire TV channels send at last one unencrypted request


In [26]:
# use host if domain doesn't exist (ip address)
requests_roku["req_domain"] = requests_roku.apply(lambda x: x.req_domain if x.req_domain else x.host, axis=1)
requests_amazon["req_domain"] = requests_amazon.apply(lambda x: x.req_domain if x.req_domain else x.host, axis=1)

### Amazon top unencrypted

In [36]:
from log_analysis import get_https_upgrade_redirectors
def get_most_common_unencrypted(crawl_name, http_req, http_resp):
    redirects, https_upgrades, cross_origin_redirects = get_https_upgrade_redirectors(
        crawl_name, http_req, http_resp)
    https_upgrade_domains = set(https_upgrades.req_domain.unique())
    df = get_popular_domains_from_reqs(http_req, 10)
    df['https_upgrade'] = df.req_domain.map(lambda x: x in https_upgrade_domains)
    # Omit the https_upgrade column if none of the domains upgrade to HTTPS
    if list(df['https_upgrade'].unique()) == [False]:
        print("None of the endpoints upgrade to HTTPS")
        del df['https_upgrade']
    return df

In [37]:
df = get_most_common_unencrypted(CrawlFireTVTop1KMITM, requests_amazon, responses_amazon)
df

/home/user/dev/smart-tv/src/analysis/notebooks/log_analysis.py:747: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  http_resp['url'] = http_resp.apply(lambda x: get_resp_url(x, req_urls), axis=1)
/home/user/dev/smart-tv/src/analysis/notebooks/log_analysis.py:506: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  lambda x: get_fld(x, fail_silently=True))


None of the endpoints upgrade to HTTPS


/home/user/dev/smart-tv/src/analysis/notebooks/log_analysis.py:756: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  and x.req_domain==x.loc_domain), axis=1)


,req_domain,Num. of channels
239,amazon-adsystem.com,678
491,scorecardresearch.com,108
383,ifood.tv,50
385,images-amazon.com,45
282,cloudinary.com,32
525,titantv.com,29
570,wsi.com,27
278,cdn01.net,24
408,lightcast.com,24
307,demdex.net,22


In [39]:
print(make_latex_table(df, label="most_common_insecure_domains_amazon",
                       caption="Most prevalent domains contacted over unencrypted connections (Amazon)"))


\begin{table}[H]
%\centering
%\resizebox{\columnwidth}{!}{%
\begin{tabular}{lr}
\toprule
 Req domain            &   Num. of channels \\
\midrule
 amazon-adsystem.com   &                678 \\
 scorecardresearch.com &                108 \\
 ifood.tv              &                 50 \\
 images-amazon.com     &                 45 \\
 cloudinary.com        &                 32 \\
 titantv.com           &                 29 \\
 wsi.com               &                 27 \\
 cdn01.net             &                 24 \\
 lightcast.com         &                 24 \\
 demdex.net            &                 22 \\
\bottomrule
\end{tabular}
%}
\caption{Most prevalent domains contacted over unencrypted connections (Amazon)}
\label{tab:most_common_insecure_domains_amazon}
\end{table}


### Roku top unencrypted

In [44]:
df = get_most_common_unencrypted(CrawlRokuTop1KMITM, requests_roku, responses_roku)
df

/home/user/dev/smart-tv/src/analysis/notebooks/log_analysis.py:747: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  http_resp['url'] = http_resp.apply(lambda x: get_resp_url(x, req_urls), axis=1)
/home/user/dev/smart-tv/src/analysis/notebooks/log_analysis.py:506: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  lambda x: get_fld(x, fail_silently=True))


None of the endpoints upgrade to HTTPS


/home/user/dev/smart-tv/src/analysis/notebooks/log_analysis.py:756: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  and x.req_domain==x.loc_domain), axis=1)


,req_domain,Num. of channels
157,doubleclick.net,266
212,google-analytics.com,175
394,scorecardresearch.com,145
380,roku.com,145
234,ifood.tv,90
453,tremorhub.com,79
419,stickyadstv.com,74
248,irchan.com,74
304,monarchads.com,73
10,1rx.io,66


In [46]:
print(make_latex_table(df, label="most_common_insecure_domains_roku",
                       caption="Most prevalent domains contacted over unencrypted connections (Roku)"))


\begin{table}[H]
%\centering
%\resizebox{\columnwidth}{!}{%
\begin{tabular}{lr}
\toprule
 Req domain            &   Num. of channels \\
\midrule
 doubleclick.net       &                266 \\
 google-analytics.com  &                175 \\
 scorecardresearch.com &                145 \\
 roku.com              &                145 \\
 ifood.tv              &                 90 \\
 tremorhub.com         &                 79 \\
 stickyadstv.com       &                 74 \\
 irchan.com            &                 74 \\
 monarchads.com        &                 73 \\
 1rx.io                &                 66 \\
\bottomrule
\end{tabular}
%}
\caption{Most prevalent domains contacted over unencrypted connections (Roku)}
\label{tab:most_common_insecure_domains_roku}
\end{table}
